---
title: Savitzky–Golay
---

<iframe width="560" height="315" src="https://www.youtube.com/embed/6mEzPkOcITk" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
order 3

<iframe width="560" height="315" src="https://www.youtube.com/embed/QpSXv8AyLzY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
order 2


In [ ]:
print("hello savgol")